# Create an ai agent assistant 

## https://platform.openai.com/docs/api-reference/threads

In [4]:
from openai import OpenAI
import os

api_key = os.getenv('OPENAI_API_KEY')  # Ensure API key is set

if not api_key:
    raise ValueError("API Key is missing! Make sure 'OPENAI_API_KEY' is set in the environment.")

client = OpenAI(api_key=api_key)

my_assistant = client.beta.assistants.create(
    instructions="You are a SQL validation master. When asked a question, write and run Python code to answer the question.",
    name="SQL Validator",
    tools=[{"type": "code_interpreter"}],  
    model="gpt-4o-mini"
)

print(my_assistant)


Assistant(id='asst_OeSF0KiAH9wCwrXtgfyNw0Cy', created_at=1741312768, description=None, instructions='You are a SQL validation master. When asked a question, write and run Python code to answer the question.', metadata={}, model='gpt-4o-mini', name='SQL Validator', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None)


# List of available models 

In [5]:
from openai import OpenAI
import os

api_key = os.getenv("thrive_ai_open_ai_key")  # Ensure API key is set correctly
client = OpenAI(api_key=api_key)

models = client.models.list()
for model in models:
    print(model.id)


gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o-mini-audio-preview-2024-12-17
dall-e-3
dall-e-2
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
o1-mini-2024-09-12
o1-mini
omni-moderation-latest
gpt-4o-mini-audio-preview
omni-moderation-2024-09-26
whisper-1
babbage-002
tts-1-hd-1106
text-embedding-3-large
tts-1-hd
o1-preview
o1-preview-2024-09-12
gpt-4o-2024-08-06
gpt-3.5-turbo-instruct-0914
gpt-4o
tts-1
tts-1-1106
davinci-002
gpt-3.5-turbo-1106
gpt-3.5-turbo-instruct
gpt-4o-2024-11-20
gpt-3.5-turbo-0125
gpt-3.5-turbo
gpt-3.5-turbo-16k
gpt-4o-mini-2024-07-18
gpt-4o-mini
text-embedding-3-small
gpt-4o-2024-05-13
text-embedding-ada-002


# Show all available assistants i've created

In [6]:
from openai import OpenAI
import os

api_key = os.getenv("OPENAI_API_KEY")  # Ensure API key is set correctly
client = OpenAI(api_key=api_key)


my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
print(my_assistants.data)


[Assistant(id='asst_OeSF0KiAH9wCwrXtgfyNw0Cy', created_at=1741312768, description=None, instructions='You are a SQL validation master. When asked a question, write and run Python code to answer the question.', metadata={}, model='gpt-4o-mini', name='SQL Validator', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0, reasoning_effort=None), Assistant(id='asst_Ft4GdC2SJhv4HCiQORGPkprA', created_at=1738434964, description=None, instructions='You are a SQL validation master. When asked a question, write and run Python code to answer the question.', metadata={}, model='gpt-4o-mini', name='SQL Validator', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_se

In [7]:
from openai import OpenAI
import os
import time  

# Get API Key & Assistant ID from Environment
api_key = os.getenv('OPENAI_API_KEY')  
assistant_id = os.getenv('thrive_ai_sql_assistant')  # Set this in your environment

if not api_key or not assistant_id:
    raise ValueError("Missing API Key or Assistant ID! Ensure they are set in the environment.")

# Initialize OpenAI Client
client = OpenAI(api_key=api_key)

# Create a Thread for conversation history (if needed)
thread = client.beta.threads.create()

def ask_assistant(question, timeout=60, max_retries=5):
    """ Sends a question to the assistant and returns the response with retry logic. """
    
    # Send the user's message
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )

    # Run the assistant
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id
    )

    # Implement exponential backoff for polling
    start_time = time.time()
    wait_time = 1  # Initial wait time
    retries = 0

    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        
        if run_status.status == "completed":
            break
        elif run_status.status in ["failed", "cancelled", "expired"]:
            raise RuntimeError(f"Assistant run failed with status: {run_status.status}")
        elif time.time() - start_time > timeout:
            raise TimeoutError("Assistant response timed out.")
        
        time.sleep(wait_time)
        retries += 1
        wait_time = min(wait_time * 2, 10)  # Exponential backoff (max 10s)
        
        if retries > max_retries:
            raise TimeoutError("Max retries exceeded while waiting for assistant response.")

    # Retrieve and return the assistant's response
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in reversed(messages.data):
        if msg.role == "assistant":
            return msg.content[0].text.value

    return "No response received from assistant."

# Example: Ask the assistant a question
try:
    question = "How do I write a SQL query to find duplicate records in a table?"
    response = ask_assistant(question, timeout=30)
    print("\nAssistant's Response:\


SyntaxError: incomplete input (1552780790.py, line 71)

In [8]:
import os
import psycopg2
import openai

# Set up API key and PostgreSQL connection details
api_key = os.getenv('thrive_ai_open_ai_key')
if not api_key:
    raise ValueError("API Key is missing! Make sure 'thrive_ai_open_ai_key' is set in the environment.")



openai.api_key = api_key

def generate_sql_query(prompt):
    """Generate SQL query using OpenAI GPT."""
    response = openai.Completion.create(
        model="gpt-4",  # or gpt-3.5-turbo
        prompt=prompt,
        temperature=0,  # To make the output more deterministic
        max_tokens=150  # Limit the length of the generated query
    )
    return response.choices[0].text.strip()

def execute_sql_query(query):
    """Executes the given SQL query and returns the results."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        return f"Error executing query: {e}"

def ask_and_execute_sql(question):
    """Ask OpenAI for a SQL query and execute it on the database."""
    sql_query = generate_sql_query(question)
    print(f"Generated SQL Query: {sql_query}")
    
    if not sql_query:
        return "No query generated."

    # Execute the SQL query
    results = execute_sql_query(sql_query)
    return f"Query:\n{sql_query}\n\nResults:\n{results}"



ValueError: API Key is missing! Make sure 'thrive_ai_open_ai_key' is set in the environment.

In [9]:

# Example: Ask the assistant to run a SQL query
try:
    question = "Find the top 5 penguins with the largest beak in the penguin table."
    response = ask_and_execute_sql(question)
    print("\nAssistant's Response:\n", response)
except Exception as e:
    print(f"Error: {e}")

Error: name 'ask_and_execute_sql' is not defined


In [3]:
import os
import psycopg2
import openai
import difflib  # For fuzzy matching

# Set up API key and PostgreSQL connection details
api_key = os.getenv('thrive_ai_open_ai_key')
if not api_key:
    raise ValueError("API Key is missing! Make sure 'thrive_ai_open_ai_key' is set in the environment.")

DB_CONFIG = {
    "dbname": "example_data_db",
    "user": "postgres",
    "password": "pgpw",
    "host": "localhost",
    "port": "5432"
}

openai.api_key = api_key

def generate_sql_query(prompt):
    """Generate SQL query using OpenAI GPT."""
    response = openai.Completion.create(
        model="gpt-4",  # or gpt-3.5-turbo
        prompt=prompt,
        temperature=0,  # To make the output more deterministic
        max_tokens=150  # Limit the length of the generated query
    )
    return response.choices[0].text.strip()

def execute_sql_query(query):
    """Executes the given SQL query and returns the results."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        return f"Error executing query: {e}"

def score_sql_query(generated_query, expected_query):
    """Scores the SQL query based on how closely it matches the expected query."""
    # Step 1: Normalize queries (remove excess whitespace)
    generated_query = " ".join(generated_query.split())
    expected_query = " ".join(expected_query.split())

    # Step 2: Exact match check (score 100 if exact match)
    if generated_query == expected_query:
        return 100

    # Step 3: Fuzzy matching score using difflib (Levenshtein distance)
    sequence_matcher = difflib.SequenceMatcher(None, generated_query, expected_query)
    similarity = sequence_matcher.ratio()

    # Return a score based on the similarity (0 to 100 scale)
    return round(similarity * 100)

def ask_and_validate_sql(question, expected_sql):
    """Ask OpenAI for a SQL query, validate it, and execute it on the database."""
    # Generate the SQL query
    generated_sql = generate_sql_query(question)
    print(f"Generated SQL Query: {generated_sql}")

    # Score the SQL query
    score = score_sql_query(generated_sql, expected_sql)
    print(f"SQL Validation Score: {score}/100")

    # Execute the SQL query
    if not generated_sql:
        return "No query generated."

    results = execute_sql_query(generated_sql)
    return f"Query:\n{generated_sql}\n\nResults:\n{results}"



In [11]:
import os
import psycopg2
import openai
import difflib  # For fuzzy matching

# Set up API key and PostgreSQL connection details
api_key = os.getenv('thrive_ai_open_ai_key')
if not api_key:
    raise ValueError("API Key is missing! Make sure 'thrive_ai_open_ai_key' is set in the environment.")

DB_CONFIG = {
    "dbname": "example_data_db",
    "user": "postgres",
    "password": "pgpw",
    "host": "localhost",
    "port": "5432"
}

openai.api_key = api_key

def generate_sql_query(prompt):
    """Generate SQL query using OpenAI GPT (new method)."""
    response = openai.chat.Completion.create(
        model= "gpt-3.5-turbo", #"gpt-4",  # or gpt-3.5-turbo
        messages=[{"role": "user", "content": prompt}],
        temperature=0,  # To make the output more deterministic
        max_tokens=150  # Limit the length of the generated query
    )
    return response['choices'][0]['message']['content'].strip()

def execute_sql_query(query):
    """Executes the given SQL query and returns the results."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        return f"Error executing query: {e}"

def score_sql_query(generated_query, expected_query):
    """Scores the SQL query based on how closely it matches the expected query."""
    # Step 1: Normalize queries (remove excess whitespace)
    generated_query = " ".join(generated_query.split())
    expected_query = " ".join(expected_query.split())

    # Step 2: Exact match check (score 100 if exact match)
    if generated_query == expected_query:
        return 100

    # Step 3: Fuzzy matching score using difflib (Levenshtein distance)
    sequence_matcher = difflib.SequenceMatcher(None, generated_query, expected_query)
    similarity = sequence_matcher.ratio()

    # Return a score based on the similarity (0 to 100 scale)
    return round(similarity * 100)

def ask_and_validate_sql(question, expected_sql):
    """Ask OpenAI for a SQL query, validate it, and execute it on the database."""
    # Generate the SQL query
    generated_sql = generate_sql_query(question)
    print(f"Generated SQL Query: {generated_sql}")

    # Score the SQL query
    score = score_sql_query(generated_sql, expected_sql)
    print(f"SQL Validation Score: {score}/100")

    # Execute the SQL query
    if not generated_sql:
        return "No query generated."

    results = execute_sql_query(generated_sql)
    return f"Query:\n{generated_sql}\n\nResults:\n{results}"

# Example: Ask the assistant to run a SQL query
try:
    question = "Find the top 5 penguins with the largest beak in the penguin table."
    # Provide the expected SQL query for validation
    expected_sql = "SELECT species, bill_length_mm FROM example_data_schema.dt_penguins ORDER BY species DESC LIMIT 5;"
    
    response = ask_and_validate_sql(question, expected_sql)
    print("\nAssistant's Response:\n", response)
except Exception as e:
    print(f"Error: {e}")


Error: 'Chat' object has no attribute 'Completion'


In [13]:
import os
import psycopg2
import openai
import difflib  # For fuzzy matching

# Set up API key and PostgreSQL connection details
api_key = os.getenv('thrive_ai_open_ai_key')
if not api_key:
    raise ValueError("API Key is missing! Make sure 'thrive_ai_open_ai_key' is set in the environment.")

DB_CONFIG = {
    "dbname": "example_data_db",
    "user": "postgres",
    "password": "pgpw",
    "host": "localhost",
    "port": "5432"
}

openai.api_key = api_key

def generate_sql_query(prompt):
    """Generate SQL query using OpenAI GPT (new method)."""
    response = openai.ChatCompletion.create(
        model= "gpt-3.5-turbo",  # or gpt-4
        messages=[{"role": "user", "content": prompt}],
        temperature=0,  # To make the output more deterministic
        max_tokens=150  # Limit the length of the generated query
    )
    return response['choices'][0]['message']['content'].strip()

def execute_sql_query(query):
    """Executes the given SQL query and returns the results."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        cursor.execute(query)
        results = cursor.fetchall()
        conn.commit()
        cursor.close()
        conn.close()
        return results
    except Exception as e:
        return f"Error executing query: {e}"

def score_sql_query(generated_query, expected_query):
    """Scores the SQL query based on how closely it matches the expected query."""
    # Step 1: Normalize queries (remove excess whitespace)
    generated_query = " ".join(generated_query.split())
    expected_query = " ".join(expected_query.split())

    # Step 2: Exact match check (score 100 if exact match)
    if generated_query == expected_query:
        return 100

    # Step 3: Fuzzy matching score using difflib (Levenshtein distance)
    sequence_matcher = difflib.SequenceMatcher(None, generated_query, expected_query)
    similarity = sequence_matcher.ratio()

    # Return a score based on the similarity (0 to 100 scale)
    return round(similarity * 100)

def ask_and_validate_sql(question, expected_sql):
    """Ask OpenAI for a SQL query, validate it, and execute it on the database."""
    # Generate the SQL query
    generated_sql = generate_sql_query(question)
    print(f"Generated SQL Query: {generated_sql}")

    # Score the SQL query
    score = score_sql_query(generated_sql, expected_sql)
    print(f"SQL Validation Score: {score}/100")

    # Execute the SQL query
    if not generated_sql:
        return "No query generated."

    results = execute_sql_query(generated_sql)
    return f"Query:\n{generated_sql}\n\nResults:\n{results}"

# Example: Ask the assistant to run a SQL query
try:
    question = "Find the top 5 penguins with the largest beak in the penguin table."
    # Provide the expected SQL query for validation
    expected_sql = "SELECT species, bill_length_mm FROM example_data_schema.dt_penguins ORDER BY bill_length_mm DESC LIMIT 5;"
    
    response = ask_and_validate_sql(question, expected_sql)
    print("\nAssistant's Response:\n", response)
except Exception as e:
    print(f"Error: {e}")


Error: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



In [6]:
pip install --upgrade openai


Note: you may need to restart the kernel to use updated packages.
